# 1. Load data and packages

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
pd.options.display.max_rows = 30
import matplotlib.pyplot as plt #if using matplotlib
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
pip install pytrends

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests>=2.0 in c:\programdata\anaconda3\lib\site-packages (from pytrends) (2.26.0)



In [4]:
from pytrends.request import TrendReq

In [5]:
pytrends = TrendReq(hl='en-US', tz=360)

states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
kw_list = ['Gas tax','motor fuels tax','sales tax', 'income tax', 'property tax']
kw_list = ['Gas tax']
kw_list = ['gas']


for s in states:
    geo_state='US-'+s
    pytrends.build_payload(kw_list, cat=0, timeframe='all', geo=geo_state, gprop='')
    aux=pytrends.interest_over_time()
    aux=aux.reset_index()
    aux['state']=s
    if s=='AK':
        df=aux.copy()
    else:
        df=df.append(aux)


In [6]:
df=df[['date', 'state',kw_list[0]]]
df

,date,state,gas
0,2004-01-01,AK,48
1,2004-02-01,AK,32
2,2004-03-01,AK,41
3,2004-04-01,AK,45
4,2004-05-01,AK,51
...,...,...,...
214,2021-11-01,WY,60
215,2021-12-01,WY,54
216,2022-01-01,WY,57
217,2022-02-01,WY,59


df2=df.melt(id_vars=['date', 'state'], 
        var_name="keyword", 
        value_name="value")
df2
df2.to_csv('Googletrends_gastax.csv') 

plt.figure(figsize=(5,5), tight_layout=True)
sns.set(font_scale=1.5)
sns.set_style(style='white')
sns.set_style("whitegrid")

g = sns.relplot(x='date', y='value', kind="line",col="state",hue="keyword", col_wrap=4, estimator=None,
            height=8, data=df2)
g.set(xlabel='Date', ylabel='Google trend')
plt.ylim(0, 100)
plt.savefig('GasTax_full.pdf')

df2['year']=df2['date'].dt.year
df2.columns

df3=df2.loc[df2['value']>0,:]
df3=df3.groupby(['state', 'year', 'keyword']).mean()
df3=df3.reset_index()
plt.figure(figsize=(5,5), tight_layout=True)
sns.set(font_scale=1.5)
sns.set_style(style='white')
sns.set_style("whitegrid")

g = sns.relplot(x='year', y='value', kind="line",col="state",hue="keyword", col_wrap=4, estimator=None,
            height=8, data=df3)
g.set(xlabel='Year', ylabel='Google trend')
plt.ylim(0, 100)
plt.xlim(2004, 2022)

plt.savefig('GasTax_full_annual.pdf')

df3.to_csv('Googletrends_gastax_annualavg.csv') 

# Repeat for list of keywords

In [7]:
import time
import random

In [14]:
phrases = pd.read_excel (r'GoogleTrendsSearchPhrases.xlsx')
kw_list=phrases.loc[:,'search phrase'].tolist()
merged=df

In [15]:
kw_list2=kw_list[31:]

In [16]:

states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
count=0
count2=0
for word in kw_list2:
    count=count+1
    print(count)
    if count==10:
        time.sleep(1800)
        count=0
    for s in states:
        print(s)
        time.sleep(random.randint(60, 120))
        geo_state='US-'+s
        keyword=list(word.split('+'))  
        while True:
            try:
                pytrends = TrendReq(hl='en-US', tz=360)
                pytrends.build_payload(keyword, cat=0, timeframe='all', geo=geo_state, gprop='')
                aux=pytrends.interest_over_time()  
                count2=0
                break
            except:
                time.sleep(random.randint(5, 15))
                count2=count2+1
                if count2==3:
                    print(word)
                    time.sleep(3600)
                continue
        aux=aux.reset_index()
        aux['state']=s
        if s=='AK':
            dfaux=aux.copy()
        else:
            dfaux=dfaux.append(aux)
    if dfaux.size>0:
        dfaux=dfaux[['date', 'state',word]]
        merged=merged.merge(dfaux, how='outer', left_on=['date', 'state'], right_on=['date', 'state']) 
    time.sleep(random.randint(60, 200))        


1
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
2
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
3
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
4
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
5
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
6
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
7
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI

TN
TX
UT
VA
VT
WA
WI
WV
WY
4
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
5
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
6
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
7
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
8
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
9
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
10
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
I

NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
7
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
8
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
9
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
10
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
1
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
2
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
3
AK
AL
AR
AZ
C

MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
10
AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY


In [17]:
merged

,date,state,gas,pipeline,electric vehicle,petroleum,refinery,frack,drill,environment,...,Democrat,Independent,tea party,libertarian,entitlement,welfare state,socialism,communism,fascism,racism
0,2004-01-01,AK,48,51,0,100,51,NaN,71,75,...,0,50,16,0,0,NaN,0,0,0,36
1,2004-02-01,AK,32,64,100,84,50,NaN,70,52,...,39,62,0,0,0,NaN,48,0,0,18
2,2004-03-01,AK,41,100,0,71,63,NaN,44,67,...,25,31,0,75,0,NaN,61,63,0,0
3,2004-04-01,AK,45,41,0,93,48,NaN,100,100,...,56,47,15,0,0,NaN,92,48,0,50
4,2004-05-01,AK,51,26,0,52,0,NaN,16,29,...,36,23,28,55,0,NaN,44,46,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11164,2021-11-01,WY,60,22,14,5,20,NaN,100,17,...,25,27,2,6,0,NaN,12,5,10,11
11165,2021-12-01,WY,54,21,0,11,20,NaN,83,16,...,12,14,2,6,8,NaN,9,9,0,15
11166,2022-01-01,WY,57,12,9,12,14,NaN,86,21,...,13,22,7,3,0,NaN,3,4,5,13
11167,2022-02-01,WY,59,29,10,12,10,NaN,65,26,...,7,21,0,3,9,NaN,10,13,17,17


In [18]:
merged.columns

Index(['date', 'state', 'gas', 'pipeline', 'electric vehicle', 'petroleum',
       'refinery', 'frack', 'drill', 'environment',
       ...
       'Democrat', 'Independent', 'tea party', 'libertarian', 'entitlement',
       'welfare state', 'socialism', 'communism', 'fascism', 'racism'],
      dtype='object', length=161)

In [19]:
df2=merged.melt(id_vars=['date', 'state'], 
        var_name="keyword", 
        value_name="value")
df2
df2.to_csv('Googletrends_keywords2.csv') 

,date,state,keyword,value
0,2004-01-01,AK,gas,48.0
1,2004-02-01,AK,gas,32.0
2,2004-03-01,AK,gas,41.0
3,2004-04-01,AK,gas,45.0
4,2004-05-01,AK,gas,51.0
...,...,...,...,...
1775866,2021-11-01,WY,racism,11.0
1775867,2021-12-01,WY,racism,15.0
1775868,2022-01-01,WY,racism,13.0
1775869,2022-02-01,WY,racism,17.0


In [20]:
data0 = pd.read_csv('Googletrends_keywords1.csv')


In [25]:
data0=data0.loc[data0['keyword']!='gas',:]

In [26]:
data0.keyword.unique()

array(['Gas tax', 'income tax', 'property tax', 'sales tax', 'royalties',
       'capital gains tax', 'estate tax', 'death tax', 'excise tax',
       'payroll tax', 'gift tax', 'inheritance tax', 'diesel price',
       'fuel costs', 'transportation', 'commuting', 'roads', 'highways',
       'subway', 'mobility', 'accessibility', 'infrastructure', 'bridge',
       'tunnel', 'ferry', 'motorcycle', 'truck', 'bus', 'energy', 'oil'],
      dtype=object)

In [23]:
df2.keyword.unique()

array(['gas', 'pipeline', 'electric vehicle', 'petroleum', 'refinery',
       'frack', 'drill', 'environment', 'clean air', 'renewable',
       'pollution', 'sustain', 'climate', 'fossil fuel', 'industry',
       'business', 'economy', 'job', 'employ', 'invest', 'dollar',
       'mortgage', 'consumer', 'profit', 'bank', 'worker', 'salaries',
       'bankruptcy', 'stock market', 'inflation', 'price', 'cash',
       'pension', 'retirement', 'debt', 'entrepreneur', 'foreclosure',
       'paycheck', 'layoff', 'growth', 'workforce', 'cost of living',
       'inequality', 'trade', 'budget', 'fund', 'tax', 'fiscal',
       'deficit', 'expenditure', 'utilities', 'wind', 'electric',
       'internet', 'phone', 'solar', 'telephone', 'telecommunication',
       'education', 'school', 'teacher', 'student', 'walkout',
       'classroom', 'textbook', 'curriculum', 'health', 'medicaid',
       'hospital', 'doctor', 'medicine', 'pharmaceutical', 'obamacare',
       'physician', 'presecription', 'sick'

In [34]:
merged2=df2.append(data0)

In [35]:
merged2.reset_index()
merged2.columns
merged2=merged2[['date', 'state', 'keyword', 'value']]

,index,date,state,keyword,value,Unnamed: 0
0,0,2004-01-01 00:00:00,AK,gas,48.0,NaN
1,1,2004-02-01 00:00:00,AK,gas,32.0,NaN
2,2,2004-03-01 00:00:00,AK,gas,41.0,NaN
3,3,2004-04-01 00:00:00,AK,gas,45.0,NaN
4,4,2004-05-01 00:00:00,AK,gas,51.0,NaN
...,...,...,...,...,...,...
2110936,335065,2021-11-01,WY,oil,51.0,335065.0
2110937,335066,2021-12-01,WY,oil,49.0,335066.0
2110938,335067,2022-01-01,WY,oil,53.0,335067.0
2110939,335068,2022-02-01,WY,oil,58.0,335068.0


Index(['date', 'state', 'keyword', 'value', 'Unnamed: 0'], dtype='object')

In [36]:
merged2.to_csv('Googletrends_keywords_full.csv') 